<a href="https://colab.research.google.com/github/DongUk-Park/Caculator/blob/main/Matrix_Factorizaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 라이브러리 설치 및 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm #정규분포 그리는 라이브러리
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

데이터셋 다운로드

In [ ]:
data_POI= pd.read_json('/content/drive/MyDrive/dataset/POI info. Dataset.json', lines=True) # training data
data_Check_in = pd.read_json('/content/drive/MyDrive/dataset/User-Check-in Dataset.json', lines=True) # test data
data_Review = pd.read_json('/content/drive/MyDrive/dataset/User-Review Dataset.json', lines=True) # training data


방문한 가게에 rating : 1 부여

In [ ]:
data_Review['rating']=1
data_Review = data_Review.drop(['text'],axis=1)

train 데이터(90%)와 test 데이터(10%)로 데이터 분할

In [ ]:
data_train, data_test = train_test_split(data_Review, test_size = 0.1, random_state = 777, shuffle = True)

In [ ]:
data_train = pd.merge(data_train,data_POI,on='business_id')
data_train

,user_id,business_id,rating,latitude,longitude,categories
0,hB2x90hzlNn9zNH9DQq-Zg,nWpIXaVS8VuAsHZYID4Lww,1,39.950171,-75.162130,"Restaurants, American (New)"
1,dhDjxQGip-s9tBv6JzhjmQ,nWpIXaVS8VuAsHZYID4Lww,1,39.950171,-75.162130,"Restaurants, American (New)"
2,OcFlOxJquFzVIjQov1K1sw,nWpIXaVS8VuAsHZYID4Lww,1,39.950171,-75.162130,"Restaurants, American (New)"
3,3IFfnwuFB-Y_0vSOlqNpSQ,nWpIXaVS8VuAsHZYID4Lww,1,39.950171,-75.162130,"Restaurants, American (New)"
4,i7zuRDllj1KTqoSHUayV_w,nWpIXaVS8VuAsHZYID4Lww,1,39.950171,-75.162130,"Restaurants, American (New)"
...,...,...,...,...,...,...
671450,a4SlYCptydyiNiihq-bdZw,UGZZVg5iOT-Q9PfoST38oQ,1,40.030022,-75.132179,"Chinese, Restaurants"
671451,RXcwwDqa9q0oM4RcTmT46Q,vkQ2Wp-lWuuJY4HOMXCUzA,1,40.008503,-75.094148,"Auto Repair, Automotive, Body Shops"
671452,yi48W8hFPwivAXFc8pxbOA,hlelvgQWzowBck_AqkSpUg,1,40.036411,-75.218791,"Bookkeepers, Tax Services, Payroll Services, P..."
671453,7-daAMnlIiRXxo5zTlXsLQ,hlelvgQWzowBck_AqkSpUg,1,40.036411,-75.218791,"Bookkeepers, Tax Services, Payroll Services, P..."


user_id와 business_id 를 int형으로 변환시켜준다

business_id를 먼저 정수형으로 변환시켜주고
이후 user_id 기준으로 정렬하여 user_id 또한 정수형으로 변환시켜준다.

이 부분이 잘 안돼서 아직 공부중에 있습니다. 

In [ ]:
'''
num = 1
business_id = data_POI.loc[0,'business_id']
for i in range(746062):
  if data_train.loc[i,'business_id'] == business_id: # 가게명이 동일하다면
    data_train.loc[i,'business_id'] = num # 동일한 숫자 부여
  else: # 가게명이 이전과 다르다면
    business_id = data_POI.loc[i,'business_id']
    num +=1
    data_train.loc[i,'business_id'] = num

data_train
'''

KeyboardInterrupt: ignored

피벗테이블로 변환, 방문하지 않은 가게는 rating : 0 설정

In [ ]:
n = 111909
subset = data_train[:n] # 램 오버 문제로 우선 1/6만 실시
data_table = subset.pivot_table(values='rating', index='user_id', columns='business_id')
data_table.fillna( 0, inplace = True )

data_table

business_id,-0TffRSXXIlBYVbb5AwfTg,-ATiAtTikuGuqvaW2O6tNA,-FMI0_EhMjwXaAgHVGuG2Q,-lVEVi3vtQ8_L9kKT0bBwQ,-v1Ia8G_G00SuygAm6koUg,0346ZLBXCnsGief7r2kmQQ,05ev984NYfimRN0UiFrxaA,0Nqa0zPHoB_GVucVMd9f6w,0R2yKDNMUztQwgo8pG4z-Q,0bvtIbpsqBJZpRKSPwIfNw,...,y7qvy9LcyK2hr_VcPBiPaQ,yIIibdtcOnGsjtvgNuBqKA,yPxNNSmPHKO3m_kq53D4Jg,ypwdVHkqBc2xBPnMQCtOMg,ytynqOUb3hjKeJfRj5Tshw,zYnYOzhlG9jPoX9ANecEUw,zuNEmJXTIAMRwRY5v-oSCg,zujdPV3HT-Y-CKE1GgkMHQ,zwl7iLwHBTdkb5-0gNgGFg,zxWeLWsv6Ebrkb-MDMbD-g
user_id,,,,,,,,,,,,,,,,,,,,,
--0kuuLmuYBe3Rmu0Iycww,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--2vR0DIsmQ6WfcSzKWigw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4_p6Z3tKadJcr9Non_Vw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--CIuK7sUpaNzalLAlHJKA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--Dz7-yZ5vMuDdlHxfBWVw,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zze77nEO2AP50S1q_qRYBA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzeRWIiPtuJNRBUcxe0Upw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zziJLt25YU6dp01sewR-IQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Matrix Factorization 구현 및 학습

In [ ]:
def matrix_factorization(R, P, Q, K, steps=10, alpha=0.0002, beta=0.02): 
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T # 전치연산
    
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0: #데이터가 있다면
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j]) # 복원한 행렬과 기존 행렬의 오차 계산

                    for k in range(K): #잠재 변수 개수만큼
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j]) # 정규화(beta)를 붙여 오버피팅문제 예방

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0: #데이터가 있다면

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2) #예측오차 계산

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:
            break

    return P, Q.T

In [ ]:
data_table = np.array(data_table)
N = len(data_table) # user_id 개수
M = len(data_table[0]) # business_id 개수
K = 3 # Features 개수

 
P = np.random.rand(N,K) # 평균0, 표준편차 1의 가우시안 표준정규분포 난수를 N * K 의 매트릭스 행렬로 생성
Q = np.random.rand(M,K) # 평균0, 표준편차 1의 가우시안 표준정규분포 난수를 M * K 의 매트릭스 행렬로 생성


nP, nQ = matrix_factorization(data_table, P, Q, K) #nP : User , nQ : Item

nR = np.dot(nP, nQ.T) # nR : 복원된 행렬

In [ ]:
nR

array([[0.76154016, 0.51727343, 0.64451513, ..., 0.86766756, 0.4100822 ,
        0.71884753],
       [0.84651388, 0.61765413, 0.67377008, ..., 0.81023581, 0.48057582,
        0.63699   ],
       [0.81043936, 0.67827978, 0.6985283 , ..., 0.761479  , 0.4861598 ,
        0.59313664],
       ...,
       [0.96257194, 0.53660316, 0.59241641, ..., 0.79495961, 0.50923382,
        0.59580547],
       [0.55163429, 0.84604849, 0.82861441, ..., 0.70500424, 0.42588247,
        0.59065126],
       [0.84486441, 1.00005582, 1.1081502 , ..., 1.17290708, 0.55998838,
        1.00820792]])

## 추천모델 입출력


데이터 전처리 과정에서 user_id와 business_id 를 정수형으로 변환하였으므로 
test 데이터로 들어오는 business_id 10개와 user_id 1개 모두 해당 정수로 받아와

위 nR matrix에서 해당 (user_id,business_id)과 같은 위치에 있는 데이터 10개를 따로 받아와 큰 순서로 정렬하여 높은 5개를 출력해주면 된다.

그러나 전처리가 아직 미숙한 관계로 코드를 작성하지 못했습니다. 추후에 계속 공부해서 작성하도록 하겠습니다.

## Precision@10 / Recall@10

In [ ]:
def _compute_precision_recall(targets, predictions, k):
    '''
		targets : 테스트 데이터 중, 사용자의 피드백이 있는 item 인덱스 리스트
		predictions : 학습 데이터에 피드백이 존재하는 item을 제외한 예측 데이터
		k : 추천 수
    '''  
    pred = predictions[:k]
    num_hit = len(set(pred).intersection(set(targets)))
    precision = float(num_hit) / len(pred)
    recall = float(num_hit) / len(targets)
    return precision, recall

In [ ]:
def evaluate_ranking(model, test, train=None, k=10):
    """
		model : 학습 모델 객체
		test : test 데이터셋, csr형식의 피벗테이블 (m*n1)
		train : train 데이터셋, csr형식의 피벗테이블 (m*n2)
		k : 추천 수
    """

    test = test.tocsr()

    if train is not None:
        train = train.tocsr()

    # ks : 서로 다른 k의 리스트
    if not isinstance(k, list):
        ks = [k]
    else:
        ks = k

    precisions = [list() for _ in range(len(ks))]
    recalls = [list() for _ in range(len(ks))]

    for user_id, row in enumerate(test):
        '''
        user_id : 고유 사용자 ID (0~m-1)
        row : 각 user에 대한 피드백이 있는 열(상품) 인덱스(튜플)
        '''

        if not len(row.indices):
            continue

        predictions = model.predict(user_id)
        predictions = predictions.argsort()

        # rated : train 피벗테이블에서 해당 user_id의 피드백이 있는 열(상품) 인덱스 리스트
        if train is not None:
            rated = set(train[user_id].indices)
        else:
            rated = []

        # 예측값(추천 상품ID=열 인덱스) 중 구매 이력(rated) 없는 예측값만 필터링
        predictions = [p for p in predictions if p not in rated]

        # test 피벗테이블에서 해당 user_id의 피드백이 있는 열(상품) 인덱스 리스트
        targets = row.indices

        # 여러 개의 k에 대한 테스트를 위해 반복문 실행
        for i, _k in enumerate(ks):
            precision, recall = _compute_precision_recall(targets, predictions, _k)
            precisions[i].append(precision) # precisions : k별 각 사용자의 precision@k 측정 리스트
            recalls[i].append(recall)   # recalls : k별 각 사용자의 recalln@k 측정 리스트

    precisions = [np.mean(np.array(i)) for i in precisions]
    recalls = [np.mean(np.array(i)) for i in recalls]

    return precisions, recalls

In [ ]:
test_table = data_test.pivot_table(values='rating', index='user_id', columns='business_id')

이 부분 또한 추후에 코드를 수정하여 precision과 recalls 측정을 다시 해보겠습니다.

In [ ]:
precisions , recalls = evaluate_ranking(nR, data_test, data_train)

AttributeError: ignored